# FRANCE TRAVEL RECOMMENDER

<img src="img/image.jpg" alt="Image" width="30%" height="30%">

## Importation des librairies

In [10]:
import pandas as pd
import requests
import json
import plotly.express as px
from fuzzywuzzy import process
import os

## Coordonnées GPS (API nominatim.org)

In [2]:
# Importation de la liste des villes
list_cities = pd.read_csv('src/list_cities.csv', header=None)[0].to_list()

# Définition des URL de l'API
start_url = 'https://nominatim.openstreetmap.org/search.php?city='
end_url = '&country=France&format=jsonv2'

In [3]:
# Récupération des données API latitude et longitude de chaque ville de list_cities

list_lon = []
list_lat = []

for city in list_cities:
    r = requests.get(f'{start_url}{city}{end_url}')
    lat = r.json()[0]['lat']
    lon = r.json()[0]['lon']
    list_lat.append(lat)
    list_lon.append(lon)
    

# Création du dataframe à partir des coordonnées par ville

df = pd.DataFrame({
    'city' : list_cities,
    'latitude' : list_lat,
    'longitude' : list_lon})

df['id'] = df.index + 1

df.to_csv('src/api_gps.csv', index=False)

In [4]:
# Chargement du dataframe à partir du fichier api_gps.csv

df = pd.read_csv('src/api_gps.csv')

display(df.head())
display(df.tail())

,city,latitude,longitude,id
0,Mont Saint Michel,48.635954,-1.511460,1
1,St Malo,48.649518,-2.026041,2
2,Bayeux,49.276462,-0.702474,3
3,Le Havre,49.493898,0.107973,4
4,Rouen,49.440459,1.093966,5


,city,latitude,longitude,id
30,Toulouse,43.604462,1.444247,31
31,Montauban,44.017584,1.354999,32
32,Biarritz,43.483252,-1.559278,33
33,Bayonne,43.494514,-1.473666,34
34,La Rochelle,46.159113,-1.152043,35


## Prévisions Météo (API openweathermap.org)

In [5]:
# URL + KEY API

# Votre clé API openweathermap.org
# api_keys = os.environ.get('API_KEY')

# Ma clé API
api_keys = '3080fdce1a3bf4f8a9b433bd4e30daa7'

start_url = 'https://api.openweathermap.org/data/2.5/forecast?'
lat_url = 'lat='
lon_url = 'lon='
end_url = f'&units=metric&appid={api_keys}'

In [6]:
# Récuperer les données API + les regrouper dans une liste

results_list = []

for city, lat, lon in zip(df['city'], df['latitude'], df['longitude']):
    r2 = requests.get(f'{start_url}{lat_url}{lat}&{lon_url}{lon}{end_url}')
    r2_json = r2.json()
    r2_json['city'] = city
    results_list.append(r2_json)
    
# Enregistrer le fichier au format JSON

with open('src/api_weather.json', 'w') as fichier_json:
    json.dump(results_list, fichier_json)

In [7]:
# Charger le fichier JSON api_gps

with open('src/api_weather.json', 'r') as fichier_json:
    results_list = json.load(fichier_json)

In [8]:
# Créer une liste par attributs

datetime = []
temperature = []
weather = []
pop = []
rain = []
city = []

for result in results_list:
    for forecast in result['list']:
        city_for = result['city']
        city.append(city_for)
        datetime_for = forecast['dt_txt']
        datetime.append(datetime_for)
        temperature_for = forecast['main']['temp']
        temperature.append(temperature_for)
        weather_for = forecast['weather'][0]['main']
        weather.append(weather_for)
        pop_for = forecast['pop']
        pop.append(pop_for)
        
        
    for forecast in result['list'][1:]:
        if 'rain' in forecast:
            rain_for = forecast['rain']['3h']
            rain.append(rain_for)
        else:
            rain.append(0)
    rain.append(0)

In [9]:
# Creation du second dataframe df_weather

data = {
    'city': city,
    'datetime': datetime,
    'temperature': temperature,
    'weather': weather,
    'pop': pop,
    'rain': rain
}

df_weather = pd.DataFrame(data)
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
df_weather

# Fusion des 2 dataframes
df_data = pd.merge(df, df_weather, on='city')

df_data

,city,latitude,longitude,id,datetime,temperature,weather,pop,rain
0,Mont Saint Michel,48.635954,-1.511460,1,2024-01-24 12:00:00,13.24,Clouds,0.0,0.00
1,Mont Saint Michel,48.635954,-1.511460,1,2024-01-24 15:00:00,12.52,Clouds,0.0,0.13
2,Mont Saint Michel,48.635954,-1.511460,1,2024-01-24 18:00:00,12.03,Rain,0.2,0.12
3,Mont Saint Michel,48.635954,-1.511460,1,2024-01-24 21:00:00,12.26,Rain,0.2,0.00
4,Mont Saint Michel,48.635954,-1.511460,1,2024-01-25 00:00:00,11.69,Clouds,0.0,0.00
...,...,...,...,...,...,...,...,...,...
1395,La Rochelle,46.159113,-1.152043,35,2024-01-28 21:00:00,9.71,Clouds,0.0,0.00
1396,La Rochelle,46.159113,-1.152043,35,2024-01-29 00:00:00,9.35,Clouds,0.0,0.00
1397,La Rochelle,46.159113,-1.152043,35,2024-01-29 03:00:00,9.23,Clouds,0.0,0.00
1398,La Rochelle,46.159113,-1.152043,35,2024-01-29 06:00:00,8.92,Clouds,0.0,0.00


In [10]:
df_data['pop_rain'] = df_data['pop'] * df_data['rain']

# Groupez les données par jour et effectuez les calculs
df_daily = df_data.groupby(['id', 'city', df_data['datetime'].dt.date]).agg(
    max_temperature=('temperature', 'max'),
    most_frequent_weather=('weather', lambda x: x.mode().iloc[0]),
    pop_rain=('pop_rain', 'sum'),
).reset_index()

# Affichez le DataFrame résultant
df_daily

,id,city,datetime,max_temperature,most_frequent_weather,pop_rain
0,1,Mont Saint Michel,2024-01-24,13.24,Clouds,0.0240
1,1,Mont Saint Michel,2024-01-25,11.83,Clouds,0.1456
2,1,Mont Saint Michel,2024-01-26,12.22,Clouds,0.7592
3,1,Mont Saint Michel,2024-01-27,8.15,Clouds,0.0000
4,1,Mont Saint Michel,2024-01-28,10.89,Clouds,0.0000
...,...,...,...,...,...,...
205,35,La Rochelle,2024-01-25,12.32,Clouds,0.0000
206,35,La Rochelle,2024-01-26,12.01,Clouds,0.0000
207,35,La Rochelle,2024-01-27,10.60,Clouds,0.0000
208,35,La Rochelle,2024-01-28,11.68,Clouds,0.0000


Si vous souhaitez enregistrer le fichier des prévisions météo quotidienne sur votre AWS S3, vous pouvez utiliser la commande suivante :

In [11]:
# # Enregistrement du fichier weather_daily au format CSV sur AWS S3

# import boto3

# # Connexion à la session S3

# session = boto3.Session(profile_name='default')
# s3 = session.resource("s3")
# print(s3)

# bucket_name = YOUR_BUCKET_NAME
# bucket = s3.Bucket(bucket_name)

# # Créer le fichier CSV dans le bucket
# filepath = 'YOUR_PATH/weather_daily.csv'

# put_object = bucket.put_object(
#     Key=filepath, 
#     Body=df_daily.to_csv(index=False))

# print(f'Le fichier a été créé dans le bucket au chemin : {filepath}')

In [12]:
# Groupez les données par city

df_unique = df_data.groupby(['id', 'city']).agg(
    max_temperature=('temperature', 'max'),
    most_frequent_weather=('weather', lambda x: x.mode().iloc[0]),
    pop_rain=('pop_rain', 'sum'),
).round({'pop_rain': 0}).reset_index()

# Fusion pour récupérer latitude et longitude

df_unique = pd.merge(df_unique, df, on='city').iloc[:,:-1]
df_unique = df_unique.rename(columns={'id_x': 'id'})
df_unique.head()

,id,city,max_temperature,most_frequent_weather,pop_rain,latitude,longitude
0,1,Mont Saint Michel,13.24,Clouds,1.0,48.635954,-1.511460
1,2,St Malo,12.37,Clouds,1.0,48.649518,-2.026041
2,3,Bayeux,12.68,Clouds,0.0,49.276462,-0.702474
3,4,Le Havre,11.14,Clouds,1.0,49.493898,0.107973
4,5,Rouen,12.18,Clouds,0.0,49.440459,1.093966


Si vous souhaitez enregistrer le fichier des prévisions météo par ville sur votre AWS S3, vous pouvez utiliser la commande suivante :

In [13]:
# # Enregistrement du fichier weather_cities au format CSV sur AWS S3

# import boto3

# # Connexion à la session S3

# session = boto3.Session(profile_name='default')
# s3 = session.resource("s3")
# print(s3)

# bucket_name = YOUR_BUCKET_NAME
# bucket = s3.Bucket(bucket_name)

# # Créer un fichier dans le bucket
# filepath = 'YOUR_PATH/weather_cities.csv'

# put_object = bucket.put_object(
#     Key=filepath, 
#     Body=df_daily.to_csv(index=False))

# print(f'Le fichier a été créé dans le bucket au chemin : {filepath}')

In [14]:
df_unique

,id,city,max_temperature,most_frequent_weather,pop_rain,latitude,longitude
0,1,Mont Saint Michel,13.24,Clouds,1.0,48.635954,-1.511460
1,2,St Malo,12.37,Clouds,1.0,48.649518,-2.026041
2,3,Bayeux,12.68,Clouds,0.0,49.276462,-0.702474
3,4,Le Havre,11.14,Clouds,1.0,49.493898,0.107973
4,5,Rouen,12.18,Clouds,0.0,49.440459,1.093966
5,6,Paris,13.13,Clouds,0.0,48.858890,2.320041
6,7,Amiens,12.27,Clouds,0.0,49.894171,2.295695
7,8,Lille,11.41,Clouds,0.0,50.636565,3.063528
8,9,Strasbourg,12.90,Clouds,3.0,48.584614,7.750713
9,10,Chateau du Haut Koenigsbourg,9.54,Clouds,5.0,48.249523,7.345492


In [15]:
# Création de la mapbox avec la température maximale par ville

df_unique['inverse_pop_rain'] = df_unique['pop_rain'].max() - df_unique['pop_rain'] + 1

fig = px.scatter_mapbox(
    df_unique, 
    lat="latitude", 
    lon="longitude", 
    color="max_temperature",
    text="city",
    title="Mapbox de la température maximale",
    size="inverse_pop_rain",
    size_max = 25,
    color_continuous_scale='Bluered',
)

fig.update_layout(
    mapbox_style="carto-positron", 
    mapbox_zoom=5, 
    mapbox_center={"lat": 46.603354, "lon": 1.888334},
    width=1000,
    height=800,
)

fig.show()

> Critères des top 5 destinations : 

1) Peu de pluies (moins de 10mm de précipitations)

2) Température max la plus élevée

In [16]:
# TOP 5 destinations

df_best_cities = df_unique[df_unique['pop_rain'] < 10].sort_values('max_temperature', ascending=False).iloc[0:5,:-3]

# Création d'une variable avec les 5 villes :
best_cities_list = df_best_cities['city'].tolist()

df_best_cities

,id,city,max_temperature,most_frequent_weather,pop_rain
27,28,Collioure,19.85,Clouds,0.0
29,30,Ariege,19.12,Clouds,0.0
28,29,Carcassonne,18.51,Clouds,0.0
24,25,Nimes,18.06,Clouds,0.0
33,34,Bayonne,17.99,Clouds,0.0


## Hotels (scraping booking.com)

/!\ La partie scraping peut ne pas être fonctionnelle au moment où vous testez. La raison étant les changements fréquents qu'il peut y avoir sur le site **booking.com** et qui ne sont pas pris en compte dans mon script.

N'hésitez pas à utiliser vos connaissances pour modifier les scripts.

### Scraping de la liste des hôtels par ville

In [17]:
# Liste des URL booking

start_url = 'https://www.booking.com/searchresults.fr.html?ss='

end_url = '&lang=fr&sb=1&src_elem=sb&src=searchresults&checkin=2023-11-01&checkout=2023-11-02&group_adults=2&no_rooms=1&group_children=0'

booking_url = [f'{start_url}{city.replace(" ", "+")}{end_url}' for city in list_cities]
 
# Création du fichier txt avec les URL
   
with open('src/url_cities_booking.txt', "w") as file:
    for item in booking_url:
        file.write(str(item) + "\n")

Après avoir créé les URL, il faut effectuer le premier scraping, en lançant le fichier ```scrapy_booking_cities.py```

In [18]:
# !python scrapy_booking_cities.py

In [19]:
# Charger le fichier JSON booking_cities_scraping

with open('src/booking_cities_scraping.json', 'r') as fichier_json:
    booking_cities = json.load(fichier_json)

In [20]:
# Creation de la liste en supprimant les vides, récupérer l'URL et correction des caractères de City

booking_cities_updated = []

for list in booking_cities:
    if list['hotel_name'] is not None and list['score_given'] is not None:
        list['url_hotel'] = list['url_hotel'].split('=')[1].split('"')[1]
        list['city'] = list['city'].split(":")[0].strip()
        booking_cities_updated.append(list)

In [21]:
# Création du dataframe df_booking 

df_booking = pd.DataFrame(booking_cities_updated)

df_booking = df_booking.drop_duplicates('hotel_name')

df_booking['score_given'] = df_booking['score_given'].str.replace(',', '.').astype(float)

# Fonction qui recherche la meilleure correspondance du nom de la ville par rapport à la liste des villes initiale
def get_best_match(city_name):
    # Exception pour St Malo (qui est en échec)
    if city_name == "Saint-Malo":
        return "St Malo"
    best_match, score = process.extractOne(city_name, list_cities)
    return best_match

# Application de la fonction au dataframe
df_booking['city'] = df_booking['city'].apply(get_best_match)

df_booking = df_booking.reset_index(drop=True)

df_booking

,city,hotel_name,url_hotel,score_given
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?aid,8.0
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2
3,Mont Saint Michel,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,7.3
4,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.0
...,...,...,...,...
976,Biarritz,Sofitel Biarritz Le Miramar Thalassa,https://www.booking.com/hotel/fr/thalassa-mira...,7.8
977,Biarritz,Hôtel Mirano,https://www.booking.com/hotel/fr/mirano.fr.htm...,8.7
978,La Rochelle,Résidence New Rochelle,https://www.booking.com/hotel/fr/residence-du-...,7.6
979,La Rochelle,Maison des Ambassadeurs,https://www.booking.com/hotel/fr/la-residence-...,8.7


In [22]:
# Création du fichier txt avec les URL des Hotels
   
with open('src/url_hotels_booking.txt', "w") as file:
    for item in df_booking['url_hotel']:
        file.write(str(item) + "\n")

Après avoir créé les URL des hôtels, il faut effectuer le second scraping, en lançant le fichier ```scrapy_booking_hotels.py```

In [23]:
# !python scrapy_booking_hotels.py

In [24]:
# Charger le fichier JSON booking_hotels_scraping

with open('src/booking_hotels_scraping.json', 'r') as fichier_json:
    booking_hotels = json.load(fichier_json)

In [25]:
# Traitement des données du fichier JSON

booking_hotels_updated = []

for list in booking_hotels:
    # Récupération du nom de l'hotel si dans une autre colonne
    if list['hotel_name'] is None and list['hotel_name_2'] is None:
        list['hotel_name'] = list['hotel_name_3']
    elif list['hotel_name'] is None:
        list['hotel_name'] = list['hotel_name_2']
    
    # Récupération de lat_lon si dans une autre colonne   
    if list['lat_lon'] is None and list['lat_lon_2'] is None:
        list['lat_lon'] = list['lat_lon_3']
    elif list['lat_lon'] is None:
        list['lat_lon'] = list['lat_lon_2']
    
    # Récupération de la description de l'hotel si dans une autre colonne
    if list['hotel_description'] is None and list['hotel_description_2'] is None:
        list['hotel_description'] = list['hotel_description_3']
    elif list['hotel_description'] is None:
        list['hotel_description'] = list['hotel_description_2']
        
    if list['hotel_name'] is not None:
        # Récupération latitude et longitude
        start_index = list['lat_lon'].find('data-atlas-latlng="')
        substring = list['lat_lon'][start_index:]
        end_index = substring.find('"', len('data-atlas-latlng="'))
        list['lat_lon'] = substring[len('data-atlas-latlng="'):end_index]
        
        # Split de la colonne en lat et lon
        list['lat'] = list['lat_lon'].split(',')[0]
        list['lon'] = list['lat_lon'].split(',')[1]
        
        # Mise à jour du dictionnaire des hotels
        booking_hotels_updated.append({
        'hotel_name': list['hotel_name'],
        'lat': list['lat'],
        'lon': list['lon'],        
        'hotel_description': list['hotel_description']
        })

In [26]:
# Création du dataframe booking hotels

df_booking_hotels = pd.DataFrame(booking_hotels_updated)

df_booking_hotels['lat'] = df_booking_hotels['lat'].astype(float)
df_booking_hotels['lon'] = df_booking_hotels['lon'].astype(float)

df_booking_hotels

,hotel_name,lat,lon,hotel_description
0,Mercure Mont Saint Michel,48.614247,-1.510545,Installé dans des espaces verts à seulement 2 ...
1,Les Terrasses Poulard,48.635349,-1.510379,Occupant 2 bâtiments différents au cœur du Mon...
2,Hôtel Vert,48.614700,-1.509617,L’Hotel Vert vous propose des chambres décorée...
3,Le Relais Saint Michel,48.617587,-1.510396,Le Relais Saint Michel vous accueille face à l...
4,Le Relais Du Roy,48.616263,-1.510906,Le Relais Du Roy est un hôtel 3 étoiles situé ...
...,...,...,...,...
954,Résidence Biarritz Ocean,43.480406,-1.557961,Résidence Biarritz Ocean is located in the hea...
955,Résidence New Rochelle,46.140690,-1.166160,La Résidence New Rochelle est située près du p...
956,Sofitel Biarritz Le Miramar Thalassa,43.487848,-1.554445,Situé à quelques pas du centre-ville de Biarri...
957,Maison des Ambassadeurs,46.162826,-1.151102,"Situé au cœur de La Rochelle, dans un bâtiment..."


In [27]:
# Fusion des 2 dataframes de Booking :

df_booking_merged = pd.merge(df_booking, df_booking_hotels, on='hotel_name')

# Fusion avec df_unique pour récupérer l'id de la ville

df_booking_merged = pd.merge(df_booking_merged, df_unique, on='city').iloc[:,:-6]

df_booking_merged

,city,hotel_name,url_hotel,score_given,lat,lon,hotel_description,id
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?aid,8.0,48.614700,-1.509617,L’Hotel Vert vous propose des chambres décorée...,1
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,48.635349,-1.510379,Occupant 2 bâtiments différents au cœur du Mon...,1
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,48.614247,-1.510545,Installé dans des espaces verts à seulement 2 ...,1
3,Mont Saint Michel,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,7.3,48.636063,-1.511457,La Vieille Auberge vous accueille dans le vill...,1
4,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.0,48.616263,-1.510906,Le Relais Du Roy est un hôtel 3 étoiles situé ...,1
...,...,...,...,...,...,...,...,...
954,Avignon,"The Originals Boutique, Hôtel Danieli, Avignon",https://www.booking.com/hotel/fr/hoteldanieli....,7.3,43.947566,4.806030,"L'The Originals Boutique, Hôtel Danieli, Avign...",23
955,Avignon,"Au cœur des Papes, dans bâtisse de charme stud...",https://www.booking.com/hotel/fr/au-coeur-des-...,8.7,43.946171,4.810189,Situé à moins de 200 mètres du centre d'Avigno...,23
956,Avignon,L'Observance Bed & Breakfast,https://www.booking.com/hotel/fr/l-39-observan...,8.9,43.944075,4.800156,"Situé à Avignon, l'Observance Bed & Breakfast ...",23
957,Avignon,City Résidence Avignon (5 Epi),https://www.booking.com/hotel/fr/residence-5-e...,7.3,43.943507,4.822365,Située à 10 minutes à pied des remparts et à 2...,23


In [28]:
# Récupération des 20 meilleurs hotels dans la zone géographique des 5 meilleurs destinations

df_best_hotels = df_booking_merged[df_booking_merged['city'].isin(best_cities_list)]\
    .sort_values(by = 'score_given', ascending=False)\
    .iloc[0:20,:]\
    .loc[:,['hotel_name', 'score_given', 'lat', 'lon']]
    
df_best_hotels

,hotel_name,score_given,lat,lon
479,"Domaine de la JASSO, au pied de la Cité Médiévale",10.0,43.203730,2.346508
360,Nimes: Appartement l'Auguste Centre-Ville.,10.0,43.837654,4.364321
665,"L'Escale de Collioure - Climatisé, parking pri...",10.0,42.521823,3.093709
478,"L'écrin de la Cité, Coeur de Vignes",10.0,43.202128,2.361453
477,Villa Le jardin de la cité,10.0,43.206161,2.370974
473,Maison Thérèse by Panorama Locations - SPA pri...,9.7,43.205619,2.361485
458,Carcassonne Bed and Breakfast du Palais,9.7,43.213599,2.357081
451,"Au point du jour - Gîtes Carcassonne, Lac de l...",9.7,43.178580,2.414622
347,Mazet Pierre de Vers - Le Mas des Olivers Nîmes,9.7,43.858731,4.392550
502,Le Gîte des Trois Seigneurs,9.6,42.857052,1.558233


## Mapbox des 20 meilleurs hôtels parmi les 5 meilleurs destinations

In [29]:
# Création de la mapbox avec les 20 meilleurs hotels

fig = px.scatter_mapbox(
    df_best_hotels, 
    lat="lat", 
    lon="lon", 
    text='hotel_name',
    color="score_given",
    title="Mapbox des 20 hotels les mieux notés",
    color_continuous_scale='Bluered',
    size="score_given",
)

fig.update_layout(
    mapbox_style="carto-positron", 
    width=1200,
    height=700,
)

fig.show()

Si vous souhaitez enregistrer le résultat du scraping pour chaque ville sur votre AWS S3, vous pouvez utiliser la commande suivante :

In [30]:
# # Enregistrement des fichiers {cities}_booking au format CSV sur AWS S3

# import boto3

# # Connexion à la session S3

# session = boto3.Session(profile_name='default')
# s3 = session.resource("s3")
# print(s3)

# bucket_name = YOUR_BUCKET_NAME
# bucket = s3.Bucket(bucket_name)

# # Boucle for pour la création des fichiers
# for city in list_cities[0:2]:
    
#     # Filtre du dataframe par ville
#     df_csv = df_booking_merged[df_booking_merged['city'] == city]
    
#     # Suppression des retours à la ligne
#     df_csv = df_csv.copy()
#     df_csv['hotel_description'] = df_csv['hotel_description'].str.replace('\n', ' ')
    
#     # Créer des fichiers dans le bucket
#     filepath = f"YOUR_PATH/cities_booking/{city.replace(' ', '_')}.csv"
#     put_object = bucket.put_object(
#         Key=filepath, 
#         Body=df_csv.to_csv(index=False))
#     print(f'Le fichier a été créé dans le bucket au chemin : {filepath}')

## Création de tables PostgreSQL

In [15]:
# Import sqlalchemy
from sqlalchemy import create_engine, text, MetaData, Table

# Vos informations de connexion Postgres
# DBHOST = os.environ.get("DBHOST")
# DBUSER = os.environ.get("DBUSER")
# DBPASS = os.environ.get("DBPASS")
# DBNAME = os.environ.get("DBNAME")
# PORT = "5432"

# Mes infos Postgres
DBHOST = "database-1.cxvdy8gdvp12.us-east-1.rds.amazonaws.com"
DBUSER = "postgres"
DBPASS = "12345678"
DBNAME = "postgres"
PORT = "5432"

engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

# Connect to the db
conn = engine.connect()

2024-01-24 11:17:25,345 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-01-24 11:17:25,345 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-24 11:17:25,521 INFO sqlalchemy.engine.Engine select current_schema()
2024-01-24 11:17:25,522 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-24 11:17:25,697 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-01-24 11:17:25,698 INFO sqlalchemy.engine.Engine [raw sql] {}


In [32]:
# # Création de la table Weather_daily sur le Postgres RDS

# import boto3

# # Connexion à la session S3
# session = boto3.Session(profile_name='default')
# s3 = session.resource("s3")

# bucket_name = 'jedha-bucket-clementb'
# bucket = s3.Bucket(bucket_name)

# s3_path = f's3://{bucket_name}/project_kayak/weather_daily.csv'

# # Récupération du fichier depuis S3
# df = pd.read_csv(s3_path)

# # Création de la table SQL depuis le fichier CSV importé de S3
# df.to_sql('weather_daily', engine, index=False)

In [12]:
# # Création de la table Cities-booking sur le Postgres RDS

# import boto3

# # Connexion à la session S3
# session = boto3.Session(profile_name='default')
# s3 = session.resource("s3")

# bucket_name = 'jedha-bucket-clementb'
# bucket = s3.Bucket(bucket_name)

# # Extraction des fichiers du bucket dans une liste
# prefix = 'project_kayak/cities_booking/'
# csv_files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]

# # Créer la table SQL (et la remplacer si elle est déjà existante)
# df = pd.read_csv(f's3://{bucket_name}/{csv_files[0]}')
# df.head(0).to_sql('cities_booking', engine, index=False, if_exists='replace')

# # Boucle for pour lancer la création et ajout dans la database RDS
# for csv_file in csv_files:
#     s3_path = f's3://{bucket_name}/{csv_file}'

#     # Load data from CSV into DataFrame
#     df = pd.read_csv(s3_path)
    
#     # Use the to_sql method to insert data into the database
#     # Replace 'your_table_name' with the actual table name
#     df.to_sql('cities_booking', engine, index=False, if_exists='append')

2024-01-24 11:15:12,842 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 11:15:12,847 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-01-24 11:15:12,848 INFO sqlalchemy.engine.Engine [generated in 0.00070s] {'table_name': 'cities_booking', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-01-24 11:15:13,026 INFO sqlalchemy.engine.Engine 
CREATE TABLE cities_booking (
	city TEXT, 
	hotel_name TEXT, 
	url_hotel TEXT, 
	score_given FLOAT(53), 
	lat FLOAT(53), 
	lon FLOAT(53), 
	hotel_description TEXT, 
	id B

In [16]:
# Affichage de la table weather_daily (à partir du RDS)

conn.rollback()

stmt = text("SELECT * "
            "FROM weather_daily "
            "LIMIT 5")
pd.read_sql_query(stmt, conn)

2024-01-24 11:17:28,934 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 11:17:28,934 INFO sqlalchemy.engine.Engine SELECT * FROM weather_daily LIMIT 5
2024-01-24 11:17:28,935 INFO sqlalchemy.engine.Engine [generated in 0.00098s] {}


,id,city,datetime,max_temperature,most_frequent_weather,pop_rain
0,1,Mont Saint Michel,2023-11-02,11.97,Rain,9.5584
1,1,Mont Saint Michel,2023-11-03,12.42,Rain,2.1129
2,1,Mont Saint Michel,2023-11-04,13.39,Rain,11.4685
3,1,Mont Saint Michel,2023-11-05,12.78,Rain,5.8715
4,1,Mont Saint Michel,2023-11-06,12.44,Rain,2.2328


In [17]:
# Affichage de la table cities_booking, filtré sur id = 1 (à partir du RDS)

conn.rollback()

stmt = text("SELECT * "
            "FROM cities_booking "
            "WHERE id = 1 "
            "LIMIT 5")
pd.read_sql_query(stmt, conn)

2024-01-24 11:17:29,680 INFO sqlalchemy.engine.Engine ROLLBACK
2024-01-24 11:17:29,772 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 11:17:29,773 INFO sqlalchemy.engine.Engine SELECT * FROM cities_booking WHERE id = 1 LIMIT 5
2024-01-24 11:17:29,773 INFO sqlalchemy.engine.Engine [generated in 0.00135s] {}


,city,hotel_name,url_hotel,score_given,lat,lon,hotel_description,id
0,Mont Saint Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?aid,8.0,48.614700,-1.509617,L’Hotel Vert vous propose des chambres décorée...,1
1,Mont Saint Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,7.4,48.635349,-1.510379,Occupant 2 bâtiments différents au cœur du Mon...,1
2,Mont Saint Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,8.2,48.614247,-1.510545,Installé dans des espaces verts à seulement 2 ...,1
3,Mont Saint Michel,La Vieille Auberge,https://www.booking.com/hotel/fr/la-vieille-au...,7.3,48.636063,-1.511457,La Vieille Auberge vous accueille dans le vill...,1
4,Mont Saint Michel,Le Relais Du Roy,https://www.booking.com/hotel/fr/le-relais-du-...,8.0,48.616263,-1.510906,Le Relais Du Roy est un hôtel 3 étoiles situé ...,1
